**EXTRACCIÓN Y LIMPIEZA DE DATOS**

**1. EXTRACCIÓN DE DATOS**

In [1]:
# importación de librerias

import pandas as pd


import numpy as np

import warnings
warnings.filterwarnings ('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import seaborn as sns

import matplotlib.pyplot as plt

In [2]:

from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver
from selenium.webdriver.common.by import By

In [3]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
#opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
#opciones.add_argument('--incognito')

1.1 Importación CSV de Base de datos de Registro de Servicios Sociales de la Comunidad de Madrid

 - **Fuente**: 'https://datos.comunidad.madrid/catalogo/dataset/servicios_sociales_registro_centros'
 - **Método de extracción**: descarga CSV de la página web oficial.

In [4]:
#import csv and creat a copy of the original df

ssmadrid_ori = pd.read_csv('../ETL-Project/data/servicios_sociales_registro_centros.csv',
                                  encoding = 'latin1', on_bad_lines='skip',sep=';')

ssmadrid = ssmadrid_ori.copy()

In [5]:
ssmadrid.head(1)

,numero_registro,denominacion,plazas_autorizadas_numero,plazas_descripcion,tipologia,subtipo,sector,municipio_codigo,municipio_nombre,direccion_via_tipo,direccion_via_codigo,direccion_via_nombre,direccion_portal,direccion_coor_x,direccion_coor_y,direccion_codigo_postal,contacto_telefono,entidad_denominacion,entidad_cif,entidad_naturaleza
0,C0003,CENTRO DE REHABILITACION LABORAL NUEVA VIDA,40,Capacidad máxima simultánea,CENTROS DE REHABILITACIÓN,CENTROS DE REHABILITACIÓN: OTROS,Personas con enfermedad mental crónica,079,Madrid,CALLE,01394,de Colomer,14,443535,4475980,28028,91355 26 80,ASOCIACION PSIQUIATRIA Y VIDA,G28727261,PRIVADA SIN ÁNIMO DE LUCRO


1.2 Scraping codigos postales de la comunidad de Madrid

- **Fuente**: 'https://www.worldpostalcodes.org/es/espana/comunidad-autonoma/lista-de-codigos-postales-en-community-of-madrid'
- **Método**: scraping utilizando la herramienta Selenium


In [6]:
PATH = ChromeDriverManager().install()     # instala el driver de chrome

driver=webdriver.Chrome(PATH, options=opciones)      # abre una venta una de chrome


In [7]:
url = 'https://www.worldpostalcodes.org/es/espana/comunidad-autonoma/lista-de-codigos-postales-en-community-of-madrid'

In [8]:
driver.implicitly_wait(3)
driver.get(url)

In [9]:
tabla = driver.find_elements(By.XPATH, '/html/body/div/div/div/div/table[1]')

litatabla = [e.text.strip().split('\n') for e in tabla]




In [10]:
columnas=driver.find_elements(By.TAG_NAME, 'th') 

listcols= [e.text.strip() for e in columnas]

listcols

['Codigo Postal',
 'Nombre del lugar del Código Postal',
 'Comunidad Autónoma',
 'Provincia']

In [11]:
data = [[e.text for e in f.find_elements(By.TAG_NAME, 'td')]  
        for f in driver.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')]

data = data[1:]

In [12]:
cp_madrid = pd.DataFrame(data, columns = listcols)

cp_madrid.head(1)

,Codigo Postal,Nombre del lugar del Código Postal,Comunidad Autónoma,Provincia
0,28001,Madrid,Comunidad de Madrid,Comunidad de Madrid


In [13]:
cp_madrid.columns = [e.lower(). replace(' ', '_') for e in cp_madrid]

cp_madrid.head()

,codigo_postal,nombre_del_lugar_del_código_postal,comunidad_autónoma,provincia
0,28001,Madrid,Comunidad de Madrid,Comunidad de Madrid
1,28002,Madrid,Comunidad de Madrid,Comunidad de Madrid
2,28003,Madrid,Comunidad de Madrid,Comunidad de Madrid
3,28004,Madrid,Comunidad de Madrid,Comunidad de Madrid
4,28005,Madrid,Comunidad de Madrid,Comunidad de Madrid


In [14]:
cp_madrid.to_csv('../ETL-Project/data/cp_madrid.csv', index = False)

1.3 Población por cada código postal de la Comunidad de Madrid de 2022

- **Fuente**: INE = 'https://www.ine.es/jaxiT3/Tabla.htm?t=2881'
- **Método**: configuración de la selección de datos (población por cada CP de la Comunidad de Madrid del año 2022, por género y número total de personas)

1.4 Ayudas Sociales de la Comunidad de Madrid de 2022

- **Fuente**: Portal de datos abiertos del Ayuntamiento de Madrid = 'https://datos.madrid.es/portal/site/egob'
- **Método**: descarga de CSV

1.5 Panel de indicadores de distritos y barrios de Madrid de 2022.
- **Fuente**: Portal de datos abiertos del Ayuntamiento de Madrid = 'https://datos.madrid.es/portal/site/egob'
- **Método**: descarga de documento Excel.

**2. EXPLORACIÓN Y LIMPIEZA DE DATOS**

Limpieza y unificación de las tablas de población y códigos postales -> poblacion_completa

In [15]:
#En primer lugar, trataremos de unificar en un sólo dataframe tanto la tabla de códigos postales de Madrid, como la tabla de población por código postal. 

poblacion_ori = pd.read_csv('../ETL-Project/data/poblacion_cp.csv',
                                  encoding = 'latin1', on_bad_lines='skip',sep='\t')

poblacion = poblacion_ori.copy()

In [16]:
poblacion.shape

(540, 4)

In [17]:
poblacion.head()

,Municipios,Sexo,Periodo,Total
0,28 Madrid,Total,2022,6.750.336
1,28 Madrid,Hombres,2022,3.230.154
2,28 Madrid,Mujeres,2022,3.520.182
3,"28001 Acebeda, La",Total,2022,62
4,"28001 Acebeda, La",Hombres,2022,32


In [18]:
#las primeras tres filas, representan los totales de población. Aunque sea un dato importante a priori, pueden distorsionar los datos posteriormente y siempre podremos calcular el total mediante queries, por lo que eliminaremos estas filas. 

poblacion = poblacion.drop(poblacion.index[:3])

poblacion.head()

,Municipios,Sexo,Periodo,Total
3,"28001 Acebeda, La",Total,2022,62
4,"28001 Acebeda, La",Hombres,2022,32
5,"28001 Acebeda, La",Mujeres,2022,30
6,28002 Ajalvir,Total,2022,4.751
7,28002 Ajalvir,Hombres,2022,2.473


In [19]:
#Ahora separaremos la columna municipios en dos columnas, una con el codigo postal y otra con el nombre del Municipio. 

poblacion['codigo_postal'] = poblacion['Municipios'].str[:5]
poblacion['nombre_municipio'] = poblacion['Municipios'].str[5:]

poblacion.head()

,Municipios,Sexo,Periodo,Total,codigo_postal,nombre_municipio
3,"28001 Acebeda, La",Total,2022,62,28001,"Acebeda, La"
4,"28001 Acebeda, La",Hombres,2022,32,28001,"Acebeda, La"
5,"28001 Acebeda, La",Mujeres,2022,30,28001,"Acebeda, La"
6,28002 Ajalvir,Total,2022,4.751,28002,Ajalvir
7,28002 Ajalvir,Hombres,2022,2.473,28002,Ajalvir


In [20]:
#ahora vamos a intentar tener 3 columnas una de total otra de mujer y otra de hombre

poblacion_ordenada = poblacion.pivot(index=['codigo_postal', 'nombre_municipio'], columns='Sexo', values='Total')

poblacion_ordenada = poblacion_ordenada[['Total', 'Hombres', 'Mujeres']]

poblacion = poblacion_ordenada

poblacion_ordenada.head()

,Sexo,Total,Hombres,Mujeres
codigo_postal,nombre_municipio,,,
28001,"Acebeda, La",62,32,30
28002,Ajalvir,4.751,2.473,2.278
28003,Alameda del Valle,241,137,104
28004,"Álamo, El",10.123,4.972,5.151
28005,Alcalá de Henares,196.888,95.472,101.416


In [21]:
poblacion_ordenada.columns


Index(['Total', 'Hombres', 'Mujeres'], dtype='object', name='Sexo')

In [22]:

poblacion_ordenada = poblacion_ordenada.reset_index()

poblacion_ordenada.head()

Sexo,codigo_postal,nombre_municipio,Total,Hombres,Mujeres
0,28001,"Acebeda, La",62,32,30
1,28002,Ajalvir,4.751,2.473,2.278
2,28003,Alameda del Valle,241,137,104
3,28004,"Álamo, El",10.123,4.972,5.151
4,28005,Alcalá de Henares,196.888,95.472,101.416


In [23]:
poblacion_ordenada.columns

Index(['codigo_postal', 'nombre_municipio', 'Total', 'Hombres', 'Mujeres'], dtype='object', name='Sexo')

In [24]:
poblacion_ordenada.rename_axis(index={"Sexo": "Indice"}, inplace=True)

In [25]:
poblacion_ordenada.head()

Sexo,codigo_postal,nombre_municipio,Total,Hombres,Mujeres
0,28001,"Acebeda, La",62,32,30
1,28002,Ajalvir,4.751,2.473,2.278
2,28003,Alameda del Valle,241,137,104
3,28004,"Álamo, El",10.123,4.972,5.151
4,28005,Alcalá de Henares,196.888,95.472,101.416


In [26]:
poblacion_ordenada.columns

Index(['codigo_postal', 'nombre_municipio', 'Total', 'Hombres', 'Mujeres'], dtype='object', name='Sexo')

In [27]:
poblacion_ordenada.rename_axis(index=None, inplace=True)

In [28]:
poblacion_ordenada.head()

Sexo,codigo_postal,nombre_municipio,Total,Hombres,Mujeres
0,28001,"Acebeda, La",62,32,30
1,28002,Ajalvir,4.751,2.473,2.278
2,28003,Alameda del Valle,241,137,104
3,28004,"Álamo, El",10.123,4.972,5.151
4,28005,Alcalá de Henares,196.888,95.472,101.416


In [29]:
poblacion_ordenada.reset_index(drop=True, inplace=True)

In [30]:
poblacion_ordenada.head()

Sexo,codigo_postal,nombre_municipio,Total,Hombres,Mujeres
0,28001,"Acebeda, La",62,32,30
1,28002,Ajalvir,4.751,2.473,2.278
2,28003,Alameda del Valle,241,137,104
3,28004,"Álamo, El",10.123,4.972,5.151
4,28005,Alcalá de Henares,196.888,95.472,101.416


In [31]:
poblacion_ordenada.shape

(179, 5)

In [32]:
cpmadrid_ori = pd.read_csv('../ETL-Project/data/cp_madrid.csv',
                                  encoding = 'latin1', on_bad_lines='skip',sep=',')

cpmadrid = cpmadrid_ori.copy()

In [33]:
cpmadrid.shape

(295, 4)

In [34]:
cpmadrid.head()

,codigo_postal,nombre_del_lugar_del_cÃ³digo_postal,comunidad_autÃ³noma,provincia
0,28001,Madrid,Comunidad de Madrid,Comunidad de Madrid
1,28002,Madrid,Comunidad de Madrid,Comunidad de Madrid
2,28003,Madrid,Comunidad de Madrid,Comunidad de Madrid
3,28004,Madrid,Comunidad de Madrid,Comunidad de Madrid
4,28005,Madrid,Comunidad de Madrid,Comunidad de Madrid


In [35]:
cpmadrid.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295 entries, 0 to 294
Data columns (total 4 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   codigo_postal                        295 non-null    int64 
 1   nombre_del_lugar_del_cÃ³digo_postal  295 non-null    object
 2   comunidad_autÃ³noma                  295 non-null    object
 3   provincia                            295 non-null    object
dtypes: int64(1), object(3)
memory usage: 67.9 KB


In [36]:
poblacion_ordenada.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   codigo_postal     179 non-null    object
 1   nombre_municipio  179 non-null    object
 2   Total             179 non-null    object
 3   Hombres           179 non-null    object
 4   Mujeres           179 non-null    object
dtypes: object(5)
memory usage: 57.4 KB


In [37]:
poblacion_ordenada['codigo_postal'] = poblacion_ordenada['codigo_postal'].astype(int)

poblacion_completa = poblacion_ordenada.merge(cpmadrid, left_on='codigo_postal', right_on='codigo_postal', how='left')

In [38]:
poblacion_completa.head()

,codigo_postal,nombre_municipio,Total,Hombres,Mujeres,nombre_del_lugar_del_cÃ³digo_postal,comunidad_autÃ³noma,provincia
0,28001,"Acebeda, La",62,32,30,Madrid,Comunidad de Madrid,Comunidad de Madrid
1,28002,Ajalvir,4.751,2.473,2.278,Madrid,Comunidad de Madrid,Comunidad de Madrid
2,28003,Alameda del Valle,241,137,104,Madrid,Comunidad de Madrid,Comunidad de Madrid
3,28004,"Álamo, El",10.123,4.972,5.151,Madrid,Comunidad de Madrid,Comunidad de Madrid
4,28005,Alcalá de Henares,196.888,95.472,101.416,Madrid,Comunidad de Madrid,Comunidad de Madrid


In [39]:
poblacion_completa.columns = [e.lower(). replace(' ', '_') for e in poblacion_completa]

poblacion_completa.head(1)

,codigo_postal,nombre_municipio,total,hombres,mujeres,nombre_del_lugar_del_cã³digo_postal,comunidad_autã³noma,provincia
0,28001,"Acebeda, La",62,32,30,Madrid,Comunidad de Madrid,Comunidad de Madrid


In [40]:
poblacion_completa.drop(poblacion_completa.columns[-3:], axis=1, inplace=True)

In [41]:
poblacion_completa.head(1)

,codigo_postal,nombre_municipio,total,hombres,mujeres
0,28001,"Acebeda, La",62,32,30


In [42]:
poblacion_completa.shape

(179, 5)

In [43]:
poblacion_completa.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179 entries, 0 to 178
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   codigo_postal     179 non-null    int32 
 1   nombre_municipio  179 non-null    object
 2   total             179 non-null    object
 3   hombres           179 non-null    object
 4   mujeres           179 non-null    object
dtypes: int32(1), object(4)
memory usage: 48.5 KB


In [44]:
poblacion_completa.head()

,codigo_postal,nombre_municipio,total,hombres,mujeres
0,28001,"Acebeda, La",62,32,30
1,28002,Ajalvir,4.751,2.473,2.278
2,28003,Alameda del Valle,241,137,104
3,28004,"Álamo, El",10.123,4.972,5.151
4,28005,Alcalá de Henares,196.888,95.472,101.416


In [45]:
poblacion_completa['total'] = poblacion_completa['total'].str.replace('.', '')

In [47]:
poblacion_completa['hombres'] = poblacion_completa['hombres'].str.replace('.', '')

In [48]:
poblacion_completa['mujeres'] = poblacion_completa['mujeres'].str.replace('.', '')

In [49]:
poblacion_completa.head()

,codigo_postal,nombre_municipio,total,hombres,mujeres
0,28001,"Acebeda, La",62,32,30
1,28002,Ajalvir,4751,2473,2278
2,28003,Alameda del Valle,241,137,104
3,28004,"Álamo, El",10123,4972,5151
4,28005,Alcalá de Henares,196888,95472,101416


In [50]:
cols = ['total', 'hombres', 'mujeres']
poblacion_completa[cols] = poblacion_completa[cols].astype(int)

In [51]:
poblacion_completa.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179 entries, 0 to 178
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   codigo_postal     179 non-null    int32 
 1   nombre_municipio  179 non-null    object
 2   total             179 non-null    int32 
 3   hombres           179 non-null    int32 
 4   mujeres           179 non-null    int32 
dtypes: int32(4), object(1)
memory usage: 18.3 KB


In [52]:
poblacion_completa.to_csv('../ETL-Project/data/poblacion_completa.csv', index = False)